In [6]:
cd drive/MyDrive/gujarati-dataset/

/content/drive/MyDrive/gujarati-dataset


In [7]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install soundfile
!pip install jiwer

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-jdi4ejw6
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-jdi4ejw6
     |████████████████████████████████| 245kB 8.3MB/s 
     |████████████████████████████████| 112kB 16.1MB/s 
  Created wheel for datasets: filename=datasets-1.5.0.dev0-cp37-none-any.whl size=192875 sha256=591e48afcdea9e5dc76ca33669186e9bed81ee3b73fd48c2d8615e3aca3d4420
  Stored in directory: /tmp/pip-ephem-wheel-cache-52iqj35x/wheels/3e/af/ff/d1cdb5d0f9cff6eba2042a16b477ada497e23f1a3b6950b928
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7zobhf6x
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-7zobhf6x
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 8.2MB/s 
     |███

In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Mar 28 10:59:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from datasets import load_dataset, load_metric
common_voice_train,common_voice_test = load_dataset('csv', data_files={'train': 'train.csv','test': 'test.csv'},error_bad_lines=False,encoding='utf-8',split=['train', 'test'])

Using custom data configuration default-d5d810daf4d9e6c9


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d5d810daf4d9e6c9/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [10]:
common_voice_test

Dataset({
    features: ['path', 'sentence'],
    num_rows: 462
})

In [11]:
common_voice_train

Dataset({
    features: ['path', 'sentence'],
    num_rows: 3410
})

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,અંધારવાડીદુર ગામમાં ખાસ કરીને આદિવાસી લોકો વસે છે.
1,જેમાં અંગ્રેજી આધારિત ક્રિઓલ ભાષા બોલનારાથી માંડીને આધુનિક અંગ્રેજી સ્વરૂપ ધરાવનારી ભાષા બોલનારા લોકોનો સમાવેશ થાય છે.
2,આની વચ્ચે ઉજ્જૈનમાં વિદ્રોહ ફાટી નિકળ્યો એટલે નિર્વાસિત અશોક્ને પરત બોલાવવામાં આવ્યો.
3,અભાપુરા ગામના લોકોનો મુખ્ય વ્યવસાય ખેતી છે.
4,એક દિવસ તેને સ્વપ્ન આવ્યુ કે તેનો પુત્ર ખૂબ મોટો સમ્રાટ બનશે.
5,આ લિપિ અમેરીકાની માયાલિપિનું એક વિકસિત સ્વરૂપ છે
6,પરંપરાગત રીતે પાયો નાખવાને બદલે તારની જાળી અને સિમેન્ટ નો ઉપયોગ કરીને તળિયું બનાવવામાં આવ્યું.
7,"વળાંક પર, બહાર નીકળવાનો બીજો માર્ગ લો"
8,અરેઠી ગામમાં ખાસ કરીને આદિવાસી લોકો વસે છે.
9,આઇઝેક ન્યૂટન્સ પેપર્સ એન્ડ લેટર્સ ઓન નેચરલ ફિલોસોફી એન્ડ રિલેટેડ ડોક્યુમેન્ટ્સ


In [13]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\‘\’]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

In [14]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [15]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,અખંડ આનંદ માસિક પ્રોઢવાચક વર્ગને સાત્વિક વાંચન પૂરું પાડે છે
1,છેવટે એક્પણ શસ્ત્ર ન રહેતાં દુશાસનનો પુત્ર ગદા વડે તેનું મસ્તક કચડી નાખે છે
2,અનેક નવા કલાકારોએ અતિવાસ્તવવાદી બેનરોનો પોતાના માટે ચોક્કસપણે ઉપયોગ લીધો છે
3,પણ કૃષ્ણએ અર્જુનની પડખે રહી તેની રક્ષા કરવાનું નક્કી કર્યું
4,એક પર્યાયમાં હોવું કે બીજામાં એ કર્મોની વિચિત્રતા છે
5,પરિસ્થિતિની ગંભીરતા સમજીને તેઓ એક છેડા તરફ પોતાની રિકોઈલલેસ તોપ જીપ પર લઈને ગયા
6,આકાશવાણી થઈ અને તેમાં અર્જુન અને કૃષ્ણને વિજયી ઠરાવાયા અને ઈંદ્રને ચાલ્યા જવા જણાવ્યું
7,એટલે તે ક્ષેત્ર વિદ્રોહ માટે ઉપયોગી હતું
8,પણ તે જંગલમાં તક્ષક નામના નાગરાજ રહેતા હતાં તે ઈંદ્રના મિત્ર હતાં
9,અતુલ વાસન ભારત દેશના આંતરરાષ્ટ્રીય ક્રિકેટ રમતા ખેલાડી છે


In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [17]:
vocab_train

Dataset({
    features: ['all_text', 'vocab'],
    num_rows: 1
})

In [18]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [19]:
vocab_list

['ય',
 'ભ',
 'િ',
 'ળ',
 'ૢ',
 'ક',
 '૩',
 'ઋ',
 'ઉ',
 'ં',
 'ઁ',
 'દ',
 'ગ',
 '૨',
 'અ',
 'ઘ',
 'ા',
 'થ',
 'ટ',
 'વ',
 'ૃ',
 'ઠ',
 'ઝ',
 'ડ',
 'ૈ',
 'ૌ',
 'છ',
 'ન',
 'ૂ',
 'જ',
 'ો',
 'ે',
 'ષ',
 '૧',
 'લ',
 'ઇ',
 'l',
 'u',
 'ણ',
 'ઢ',
 'શ',
 'ઊ',
 '\u200c',
 'ૅ',
 't',
 'ઑ',
 'હ',
 'બ',
 '_',
 '૯',
 'આ',
 'ુ',
 "'",
 'ૠ',
 'ઞ',
 'ચ',
 'ર',
 'ૉ',
 'ઐ',
 'ઈ',
 'ફ',
 'ધ',
 'ત',
 'ી',
 'ખ',
 'સ',
 '૦',
 '2',
 'પ',
 '\u200d',
 '૬',
 'મ',
 'ઓ',
 'g',
 '૮',
 '0',
 '૭',
 '્',
 'e',
 '૫',
 'ઃ',
 'ઔ',
 '઼',
 'r',
 'એ',
 ' ']

In [20]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 85,
 "'": 52,
 '0': 75,
 '2': 67,
 '_': 48,
 'e': 78,
 'g': 73,
 'l': 36,
 'r': 83,
 't': 44,
 'u': 37,
 'ઁ': 10,
 'ં': 9,
 'ઃ': 80,
 'અ': 14,
 'આ': 50,
 'ઇ': 35,
 'ઈ': 59,
 'ઉ': 8,
 'ઊ': 41,
 'ઋ': 7,
 'એ': 84,
 'ઐ': 58,
 'ઑ': 45,
 'ઓ': 72,
 'ઔ': 81,
 'ક': 5,
 'ખ': 64,
 'ગ': 12,
 'ઘ': 15,
 'ચ': 55,
 'છ': 26,
 'જ': 29,
 'ઝ': 22,
 'ઞ': 54,
 'ટ': 18,
 'ઠ': 21,
 'ડ': 23,
 'ઢ': 39,
 'ણ': 38,
 'ત': 62,
 'થ': 17,
 'દ': 11,
 'ધ': 61,
 'ન': 27,
 'પ': 68,
 'ફ': 60,
 'બ': 47,
 'ભ': 1,
 'મ': 71,
 'ય': 0,
 'ર': 56,
 'લ': 34,
 'ળ': 3,
 'વ': 19,
 'શ': 40,
 'ષ': 32,
 'સ': 65,
 'હ': 46,
 '઼': 82,
 'ા': 16,
 'િ': 2,
 'ી': 63,
 'ુ': 51,
 'ૂ': 28,
 'ૃ': 20,
 'ૅ': 43,
 'ે': 31,
 'ૈ': 24,
 'ૉ': 57,
 'ો': 30,
 'ૌ': 25,
 '્': 77,
 'ૠ': 53,
 'ૢ': 4,
 '૦': 66,
 '૧': 33,
 '૨': 13,
 '૩': 6,
 '૫': 79,
 '૬': 70,
 '૭': 76,
 '૮': 74,
 '૯': 49,
 '\u200c': 42,
 '\u200d': 69}

In [21]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [22]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{"'": 52,
 '0': 75,
 '2': 67,
 '[PAD]': 87,
 '[UNK]': 86,
 '_': 48,
 'e': 78,
 'g': 73,
 'l': 36,
 'r': 83,
 't': 44,
 'u': 37,
 '|': 85,
 'ઁ': 10,
 'ં': 9,
 'ઃ': 80,
 'અ': 14,
 'આ': 50,
 'ઇ': 35,
 'ઈ': 59,
 'ઉ': 8,
 'ઊ': 41,
 'ઋ': 7,
 'એ': 84,
 'ઐ': 58,
 'ઑ': 45,
 'ઓ': 72,
 'ઔ': 81,
 'ક': 5,
 'ખ': 64,
 'ગ': 12,
 'ઘ': 15,
 'ચ': 55,
 'છ': 26,
 'જ': 29,
 'ઝ': 22,
 'ઞ': 54,
 'ટ': 18,
 'ઠ': 21,
 'ડ': 23,
 'ઢ': 39,
 'ણ': 38,
 'ત': 62,
 'થ': 17,
 'દ': 11,
 'ધ': 61,
 'ન': 27,
 'પ': 68,
 'ફ': 60,
 'બ': 47,
 'ભ': 1,
 'મ': 71,
 'ય': 0,
 'ર': 56,
 'લ': 34,
 'ળ': 3,
 'વ': 19,
 'શ': 40,
 'ષ': 32,
 'સ': 65,
 'હ': 46,
 '઼': 82,
 'ા': 16,
 'િ': 2,
 'ી': 63,
 'ુ': 51,
 'ૂ': 28,
 'ૃ': 20,
 'ૅ': 43,
 'ે': 31,
 'ૈ': 24,
 'ૉ': 57,
 'ો': 30,
 'ૌ': 25,
 '્': 77,
 'ૠ': 53,
 'ૢ': 4,
 '૦': 66,
 '૧': 33,
 '૨': 13,
 '૩': 6,
 '૫': 79,
 '૬': 70,
 '૭': 76,
 '૮': 74,
 '૯': 49,
 '\u200c': 42,
 '\u200d': 69}

In [23]:
len(vocab_dict)

88

In [24]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [25]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [26]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [27]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [28]:
processor.save_pretrained("/content/drive/MyDrive/gujarati-dataset/clip/wav2vec2-base-gujarati-demo")

In [29]:
print(common_voice_train[0])

{'path': 'guf_01063_00076624578.wav', 'sentence': 'જીવનના શરૂઆતના ગાળામાં તે સોનીનો વ્યવસાય કરતા હતા'}


In [30]:
cd clip/

/content/drive/MyDrive/gujarati-dataset/clip


In [31]:
import soundfile as sf

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [32]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names, num_proc=1)

In [33]:
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names, num_proc=1)

In [34]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [35]:
common_voice_train = common_voice_train.map(resample, num_proc=1)
common_voice_test = common_voice_test.map(resample, num_proc=1)

In [36]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train))

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [37]:
rand_int = random.randint(0, len(common_voice_train))

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: અર્જુન બીજે દિવસે કૌરવ સેનાને ચીરતો હજારો લાખો લડવૈયાને એકજ દિવસમાં મારી નાખે છે
Input array shape: (118784,)
Sampling rate: 16000


In [38]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch



In [39]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

In [40]:
from google.colab import files
common_voice_train.to_csv('commom_voice_train.csv')
common_voice_test.to_csv('commom_voice_test.csv')
files.download('commom_voice_train.csv')
files.download('commom_voice_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [42]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [43]:
wer_metric = load_metric("wer")

In [44]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [45]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model.freeze_feature_extractor()

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  output_dir="./wav2vec2-base-gujarati-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [49]:
#trainer.train()

In [50]:
model = Wav2Vec2ForCTC.from_pretrained("jaimin/wav2vec2-base-gujarati-demo").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("jaimin/wav2vec2-base-gujarati-demo")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [51]:
from datasets import load_dataset, load_metric
import re

In [52]:
wer = load_metric("wer")

In [53]:
common_voice_validation = load_dataset('csv', data_files={'test': 'validation.csv'},error_bad_lines=False,encoding='utf-8',split='test')

Using custom data configuration default-0e8284e5c4c1d98e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0e8284e5c4c1d98e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [54]:
common_voice_validation

Dataset({
    features: ['path', 'sentence'],
    num_rows: 400
})

In [55]:
!pip install torchaudio

  Using cached https://files.pythonhosted.org/packages/aa/55/01ad9244bcd595e39cea5ce30726a7fe02fd963d07daeb136bfe7e23f0a5/torchaudio-0.8.1-cp37-cp37m-manylinux1_x86_64.whl
     |████████████████████████████████| 804.1MB 21kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.8.1 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.8.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101


In [56]:
import torchaudio
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'  # TODO: adapt this list to include all special characters you removed from the data
resampler = torchaudio.transforms.Resample(48_000, 16_000)

In [57]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
	speech_array, sampling_rate = torchaudio.load(batch["path"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

In [58]:
common_voice_validation = common_voice_validation.map(speech_file_to_array_fn)

In [59]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda")).logits
    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [60]:
common_voice_validation

Dataset({
    features: ['path', 'sentence', 'speech'],
    num_rows: 400
})

In [61]:
input_dict = processor(common_voice_validation["speech"][0], sampling_rate=16000,return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits


In [62]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [63]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_validation["sentence"][0].lower())

Prediction:
તેઓ જુન ઓગણીસો એકોતતર ભારતીય સૈન્ય અકાદમી ખાતે જોડાયા અને બાદમાં સત્તરપૂના હર્સમાં કમિશન પામ્યા

Reference:
તેઓ જુન ઓગણીસ સો એકોતેર ભારતીય સૈન્ય અકાદમિ ખાતે જોડાયા અને બાદમાં સત્તર પૂના હોર્સમાં કમિશન પામ્યા


In [64]:
result = common_voice_validation.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


WER: 28.921569


In [65]:
show_random_elements(result.remove_columns(["path","speech"]))

,pred_strings,sentence
0,એ પછી એ જંગલ કે કબ્રસ્તાનમાં એકાંતવાસ કરતા અને પ્રભુદ્યાનમાં મસ્ત રહેતા,એ પછી એ જંગલ કે કબ્રસ્તાનમાં એકાંતવાસ કરતા અને પ્રભુધ્યાનમાં મસ્ત રહેતા
1,ધ ઈકોલોજિકલ થયેટર આન્ડ ધ ઈવોલ્શનરી પ્લે,ધ ઇકોલોજિકલ થીએટર એન્ડ ધ ઇવોલ્યુશનરી પ્લે
2,અલુવાસ ગામના લોકોનો મુખ્ય વ્યવસાય ખેતી છે,અલુવાસ ગામના લોકોનો મુખ્ય વ્યવસાય ખેતી છે
3,અમદાવાદથી તે ત્રણસો સિત્તર કિલોમીટ અને રાજ્યમાં પરિયટનના વિકાસ માટે સરકાર દ્વારા પસંદ કરાયેલા ચૌદદ દરિયાકિનારામાનો એક દરિયાકિનારો છે,અમદાવાદથી તે ત્રણ સો સિત્તેર કિલોમીટર અને રાજ્યમાં પર્યટનના વિકાસ માટે સરકાર દ્વારા પસંદ કરાયેલા ચૌદ દરિયાકિનારામાંનો એક દરિયાકિનારો છે
4,પચીથી નવી હલફ્રિડ નાટક મંડળીમાં તિગ્દર્શક,પછીથી નવી આલ્ફ્રેડ નાટક મંડળીમાં દિગ્દર્શક
5,તેમની પત્ની કુનતીને તેની મુક્તાવસ્થામા દુર્વાસા મૂનિ દ્વારા વરદાન મોળ્યું હતું,તેમની પત્ની કુંતીને તેની મુગ્ધાવસ્થામાં દુર્વાસા મુનિ દ્વારા વરદાન મળ્યું હતું
6,આ સિવાય મુચંઈવાળી જગ્યાઓ કે જ્યાં વરસાદ વધુ હોય કે સિંચાઈની વ્યવસ્થા હોય ત્યાં અળવીની ખેતી થઈ શકે છે,આ સિવાય ઉંચાઈવાળી જગ્યાઓ કે જ્યાં વરસાદ વધુ હોય કે સિંચાઈની વ્યવસ્થા હોય ત્યાં અળવીની ખેતી થઈ શકે છે
7,આંકેવાલીયા ગામના લોકોનો મુખ્ય વ્યવસાય ખેતી,આંકેવાલીયા ગામના લોકોનો મુખ્ય વ્યવસાય ખેતી
8,અંજા જિલ્લાનું મુખ્ય મથક હવાઇ ખાતે આવેલું છે,અંજા જિલ્લાનું મુખ્ય મથક હવાઇ ખાતે આવેલું છે
9,આંકડાના મૂડના છુરણમાં મરી પીસીને મેળવી અને બેબે રતિ વજનની ગોળીઓ બનાવવી,આંકડાના મૂળના ચૂર્ણમાં મરી પીસીને મેળવી અને બેબે રતી વજનની ગોળીઓ બનાવવી


In [66]:
input_dict = processor(common_voice_test["input_values"][0], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [67]:
common_voice_prediction = load_dataset('csv', data_files={'test': 'test.csv'},error_bad_lines=False,encoding='utf-8',split='test')

Using custom data configuration default-66011583de78665e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-66011583de78665e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [68]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_prediction["sentence"][0].lower())

Prediction:
કેટલાક ઇતિહાસકારોને આ વાર્તા સાચી લાગી પણ તે માત્ર એક માન્યતા છે

Reference:
કેટલાક ઇતિહાસકારોને આ વાર્તા સાચી લાગી પણ તે માત્ર એક માન્યતા છે.


In [69]:
input_dict = processor(common_voice_test["input_values"][1], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [70]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_prediction["sentence"][1].lower())

Prediction:
અણુશસ્ત્રોના અપ્રસારની સંધિના સભ્યો રાષ્ટ્રોની સમીકષા પરિસદની બેઠકોમાં દર પાંચ વર્ષે આ સંધિની સમીક્ષા હાથ ધરવામાં આવે છે

Reference:
અણુશસ્રોના અપ્રસારની સંધિના સભ્યો રાષ્ટ્રોની સમીક્ષા પરિષદની બેઠકોમાં દર પાંચ વર્ષે આ સંધિની સમીક્ષા હાથ ધરવામાં આવે છે.
